# Step 13 — Open Model Providers

**What we built**: 9 new provider adapters covering local inference (Ollama, vLLM), cloud inference (Together, Groq, Fireworks, DeepSeek, Mistral, HuggingFace), and self-hosted inference (HuggingFace TGI). Each gets a TOML config with model metadata and a thin adapter file.

**Why it matters**: ArcLLM started with Anthropic and OpenAI. Production agents need model diversity — local models for air-gapped federal environments, fast inference (Groq) for latency-sensitive loops, cost-optimized endpoints (Together, Fireworks) for high-volume work, and frontier open models (DeepSeek, Mistral) for specialized tasks.

**Key decisions**:
- **D-100**: Thin alias adapters — 8 of 9 providers are OpenAI-compatible, so each adapter is ~10 lines inheriting `OpenaiAdapter` with just a `name` override
- **D-101**: `api_key_required` flag in `ProviderSettings` — local providers (Ollama, vLLM) don't need API keys
- **D-102**: All 10 providers in one step (Ollama, vLLM, Together, Groq, Fireworks, DeepSeek, Mistral, HuggingFace, HuggingFace TGI)
- **D-103**: Common models pre-populated with metadata; unknown models use adapter defaults
- **D-104**: Zero cost defaults for local providers (`cost_*_per_1m = 0.0`)
- **D-105**: Mistral gets quirk overrides (tool_choice "required" → "any", extra stop reason "model_length")
- **D-106**: Separate `huggingface` (cloud Inference API) and `huggingface_tgi` (self-hosted) providers

**Key learning**: 8 of 9 providers are pure OpenAI-compatible aliases (~10 lines each). Only Mistral needed quirk overrides. The thin alias pattern proves the architecture's extensibility.

In [ ]:
# Setup: ensure arcllm is importable
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

---
## 1. The Provider Landscape

| Provider | Type | API Key | Base URL | Use Case |
|----------|------|---------|----------|----------|
| **Ollama** | Local | Optional | `http://localhost:11434` | Air-gapped, dev, privacy |
| **vLLM** | Local | Optional | `http://localhost:8000` | High-perf GPU serving |
| **Together** | Cloud | Required | `https://api.together.xyz` | Cost-optimized open models |
| **Groq** | Cloud | Required | `https://api.groq.com/openai` | Ultra-fast inference |
| **Fireworks** | Cloud | Required | `https://api.fireworks.ai/inference` | Fast + function calling |
| **DeepSeek** | Cloud | Required | `https://api.deepseek.com` | Frontier reasoning (R1) |
| **Mistral** | Cloud | Required | `https://api.mistral.ai` | EU sovereignty, vision |
| **HuggingFace** | Cloud | Required | `https://api-inference.huggingface.co/models` | Model hub inference |
| **HuggingFace TGI** | Self-hosted | Optional | `http://localhost:8080` | Self-hosted TGI server |

---
## 2. The Thin Alias Pattern

Since 8 of 9 providers use the OpenAI Chat Completions API format, each adapter is a minimal alias:

In [ ]:
import inspect
from arcllm.adapters.ollama import OllamaAdapter
from arcllm.adapters.groq import GroqAdapter
from arcllm.adapters.together import TogetherAdapter
from arcllm.adapters.deepseek import DeepseekAdapter
from arcllm.adapters.fireworks import FireworksAdapter
from arcllm.adapters.vllm import VllmAdapter
from arcllm.adapters.huggingface import HuggingfaceAdapter
from arcllm.adapters.huggingface_tgi import Huggingface_TgiAdapter

# Show how small each adapter is
for name, cls in [
    ("Ollama", OllamaAdapter),
    ("Groq", GroqAdapter),
    ("Together", TogetherAdapter),
    ("DeepSeek", DeepseekAdapter),
    ("Fireworks", FireworksAdapter),
    ("vLLM", VllmAdapter),
    ("HuggingFace", HuggingfaceAdapter),
    ("HuggingFace TGI", Huggingface_TgiAdapter),
]:
    source = inspect.getsource(cls)
    lines = len(source.strip().split('\n'))
    print(f"{name:20s} → {lines} lines (inherits OpenaiAdapter)")

In [ ]:
# Let's see a full adapter file
print("=== OllamaAdapter (complete file) ===")
print(inspect.getsource(OllamaAdapter))

That's it. The `name` property override is the only difference — everything else (request building, response parsing, tool call handling) is inherited from `OpenaiAdapter`.

### Why This Works

The OpenAI Chat Completions API has become the de facto standard. All these providers expose the same:
- Endpoint: `POST /v1/chat/completions`
- Auth: `Authorization: Bearer <key>` header
- Request body: `{model, messages, tools, temperature, max_tokens}`
- Response: `{choices[0].message, usage, model}`

The TOML config handles the differences (base_url, api_key_env, model names, pricing).

---
## 3. Convention-Based Registry (D-041, D-042)

The registry discovers adapters by naming convention — no registration needed.

In [ ]:
from arcllm.registry import _get_adapter_class, clear_cache

clear_cache()

# The convention: provider_name → arcllm.adapters.{name} → {Name.title()}Adapter
providers = [
    "ollama", "vllm", "together", "groq", "fireworks",
    "deepseek", "mistral", "huggingface", "huggingface_tgi",
]

print(f"{'Provider':20s} {'Module':40s} {'Class':30s}")
print("-" * 90)
for p in providers:
    cls = _get_adapter_class(p)
    print(f"{p:20s} arcllm.adapters.{p:23s} {cls.__name__:30s}")

print(f"\nAll {len(providers)} providers discovered by convention. Zero registration code.")

Note: `huggingface_tgi` → `"huggingface_tgi".title()` = `"Huggingface_Tgi"` → `Huggingface_TgiAdapter`. Valid Python class name, works with the convention.

---
## 4. `api_key_required` Flag (D-101)

Local providers (Ollama, vLLM, HuggingFace TGI) don't need API keys. The `api_key_required` flag in TOML controls validation.

In [ ]:
from arcllm.config import load_provider_config

# Show api_key_required for each provider
print(f"{'Provider':20s} {'api_key_required':20s} {'base_url'}")
print("-" * 80)
for p in providers:
    config = load_provider_config(p)
    print(f"{p:20s} {str(config.provider.api_key_required):20s} {config.provider.base_url}")

In [ ]:
from arcllm.exceptions import ArcLLMConfigError

# Cloud provider without API key → error
old_key = os.environ.pop("GROQ_API_KEY", None)

try:
    config = load_provider_config("groq")
    GroqAdapter(config, "llama-3.3-70b-versatile")
except ArcLLMConfigError as e:
    print(f"Cloud without key: {e}")

if old_key:
    os.environ["GROQ_API_KEY"] = old_key

# Local provider without API key → works fine
os.environ.pop("OLLAMA_API_KEY", None)  # Remove if set
config = load_provider_config("ollama")
adapter = OllamaAdapter(config, "llama3.2")
print(f"\nLocal without key: {adapter.name} ({adapter.model_name}) — no error")

The implementation in `BaseAdapter.__init__`:
```python
if config.provider.api_key_required and not api_key:
    raise ArcLLMConfigError(f"Missing environment variable '{env_var}' ...")
```

And in `OpenaiAdapter._build_headers`:
```python
if self._api_key:  # Only adds Authorization header when key exists
    headers["Authorization"] = f"Bearer {self._api_key}"
```

---
## 5. Provider TOML Configs

Each provider has a TOML file with connection settings and model metadata.

In [ ]:
# Show models per provider
for p in providers:
    config = load_provider_config(p)
    models = list(config.models.keys())
    print(f"{p:20s} → {len(models)} models: {', '.join(models[:3])}{'...' if len(models) > 3 else ''}")

In [ ]:
# Detailed view: Ollama config (local, no auth, zero cost)
config = load_provider_config("ollama")

print("=== Ollama Provider Settings ===")
print(f"  base_url:         {config.provider.base_url}")
print(f"  api_key_required: {config.provider.api_key_required}")
print(f"  default_model:    {config.provider.default_model}")

print("\n=== Ollama Models ===")
for name, meta in config.models.items():
    print(f"  {name}:")
    print(f"    context_window:  {meta.context_window:,}")
    print(f"    supports_tools:  {meta.supports_tools}")
    print(f"    cost_input:      ${meta.cost_input_per_1m}/1M tokens")

In [ ]:
# Groq config (cloud, fast inference, has pricing)
config = load_provider_config("groq")

print("=== Groq Provider Settings ===")
print(f"  base_url:         {config.provider.base_url}")
print(f"  api_key_required: {config.provider.api_key_required}")
print(f"  default_model:    {config.provider.default_model}")

print("\n=== Groq Models ===")
for name, meta in config.models.items():
    print(f"  {name}: ${meta.cost_input_per_1m}/{meta.cost_output_per_1m} per 1M in/out")

### D-104: Zero Cost Defaults for Local Providers

Local providers set `cost_*_per_1m = 0.0`. Orgs can override for GPU cost tracking. The telemetry module still logs token counts — just the dollar cost is zero.

---
## 6. TOML Gotcha — Dotted Model Names

Model names with dots (e.g., `llama3.2`) must be quoted in TOML. Unquoted dots are parsed as nested tables.

```toml
# CORRECT — quoted string key
[models."llama3.2"]
context_window = 128000

# WRONG — parsed as models.llama3.2 (nested table)
[models.llama3.2]
```

This caught us during implementation — all Ollama models with dots need quoted keys.

In [ ]:
# Verify dotted model names parse correctly
config = load_provider_config("ollama")
dotted_models = [m for m in config.models.keys() if "." in m]
print(f"Dotted model names in Ollama: {dotted_models}")

for model_name in dotted_models:
    meta = config.models[model_name]
    print(f"  {model_name}: context={meta.context_window:,}, tools={meta.supports_tools}")

---
## 7. Mistral Quirk Overrides (D-105)

Mistral is the only provider that isn't a pure OpenAI alias. It has two quirks:

In [ ]:
from arcllm.adapters.mistral import MistralAdapter, _MISTRAL_STOP_REASON_MAP

print("=== Mistral Adapter (full source) ===")
print(inspect.getsource(MistralAdapter))

### Quirk 1: `tool_choice` Mapping

```
OpenAI:  tool_choice = "required"  → forces tool use
Mistral: tool_choice = "any"       → same behavior, different keyword
```

The adapter translates `"required"` → `"any"` in `_build_request_body()`.

### Quirk 2: Extra Stop Reason

```
OpenAI:  finish_reason = "length"         → max tokens hit
Mistral: finish_reason = "model_length"   → same thing, different name
```

In [ ]:
# Mistral stop reason mapping
print("Mistral stop_reason mapping:")
for mistral_reason, arcllm_reason in _MISTRAL_STOP_REASON_MAP.items():
    print(f"  {mistral_reason:15s} → {arcllm_reason}")
print()

# Compare with OpenAI's mapping
from arcllm.adapters.openai import _STOP_REASON_MAP
print("OpenAI stop_reason mapping:")
for openai_reason, arcllm_reason in _STOP_REASON_MAP.items():
    print(f"  {openai_reason:15s} → {arcllm_reason}")

print("\nOnly difference: Mistral has 'model_length' (extra) and no 'content_filter'")

In [ ]:
# Demonstrate tool_choice translation
os.environ.setdefault("MISTRAL_API_KEY", "test-key")
config = load_provider_config("mistral")
adapter = MistralAdapter(config, "mistral-large-latest")

from arcllm.types import Message, Tool

body = adapter._build_request_body(
    messages=[Message(role="user", content="test")],
    tools=[Tool(name="calc", description="Calculator", parameters={"type": "object"})],
    tool_choice="required",
)

print(f"Input tool_choice: 'required'")
print(f"Output tool_choice: '{body.get('tool_choice')}'")
print("Translated to Mistral's 'any' keyword")

---
## 8. HuggingFace vs HuggingFace TGI (D-106)

Two separate providers for different deployment models:

In [ ]:
hf_config = load_provider_config("huggingface")
tgi_config = load_provider_config("huggingface_tgi")

print(f"{'':20s} {'HuggingFace (cloud)':30s} {'HuggingFace TGI (self-hosted)'}")
print("-" * 80)
print(f"{'base_url':20s} {hf_config.provider.base_url:30s} {tgi_config.provider.base_url}")
print(f"{'api_key_required':20s} {str(hf_config.provider.api_key_required):30s} {str(tgi_config.provider.api_key_required)}")
print(f"{'api_key_env':20s} {hf_config.provider.api_key_env:30s} {tgi_config.provider.api_key_env}")
print(f"{'default_model':20s} {hf_config.provider.default_model:30s} {tgi_config.provider.default_model}")

| Aspect | `huggingface` | `huggingface_tgi` |
|--------|--------------|-------------------|
| Hosting | HuggingFace cloud Inference API | Self-hosted TGI server |
| Auth | Required (HF token) | Optional |
| Base URL | `https://api-inference.huggingface.co/models` | `http://localhost:8080` |
| Model names | HF hub format (`meta-llama/...`) | Whatever you loaded |
| Use case | Quick prototyping, HF ecosystem | Production self-hosted |

Both use the OpenAI-compatible API format, so both are thin aliases.

---
## 9. `load_model()` for All Providers

In [ ]:
from arcllm.registry import load_model, clear_cache

# Set dummy keys for cloud providers
for env_var in ["TOGETHER_API_KEY", "GROQ_API_KEY", "FIREWORKS_API_KEY",
                "DEEPSEEK_API_KEY", "MISTRAL_API_KEY", "HF_API_TOKEN"]:
    os.environ.setdefault(env_var, "test-key")

print(f"{'Provider':20s} {'Adapter Class':30s} {'Model':40s}")
print("-" * 90)

for provider in providers:
    clear_cache()
    model = load_model(provider)
    print(f"{provider:20s} {type(model).__name__:30s} {model.model_name}")

In [ ]:
# Specify a non-default model
clear_cache()
model = load_model("ollama", model="deepseek-r1:8b")
print(f"Provider: {model.name}")
print(f"Model:    {model.model_name}")
print(f"Thinking: {model._model_meta.supports_thinking}")
print(f"Tools:    {model._model_meta.supports_tools}")

In [ ]:
# Full stack with modules on a local provider
clear_cache()
os.environ["ARCLLM_SIGNING_KEY"] = "test-key"

model = load_model(
    "ollama",
    telemetry=True,
    audit=True,
    security=True,
    retry=True,
    rate_limit=True,
)

layers = []
layer = model
while hasattr(layer, '_inner'):
    layers.append(type(layer).__name__)
    layer = layer._inner
layers.append(type(layer).__name__)
print(f"Stack: {' → '.join(layers)}")
print(f"\nAll modules work with local providers — same wrapping, same API.")

---
## 10. Fallback Chains Across Providers

With 11 providers available, fallback chains become powerful:

In [ ]:
# Example fallback chains for different use cases

chains = {
    "Cost-optimized": ["groq", "together", "fireworks"],
    "Air-gapped federal": ["ollama", "vllm"],
    "Frontier reasoning": ["deepseek", "anthropic", "openai"],
    "EU sovereignty": ["mistral", "huggingface"],
    "General purpose": ["anthropic", "openai", "groq"],
}

for name, chain in chains.items():
    print(f"{name:25s} → {' → '.join(chain)}")

print("\nUsage: load_model('groq', fallback={'chain': ['together', 'fireworks']})")

---
## 11. Model Metadata — Capabilities and Pricing

In [ ]:
# Compare pricing across providers for similar models
print("=== Llama 3.x Model Pricing (per 1M tokens) ===")
print(f"{'Provider':15s} {'Model':40s} {'Input':>8s} {'Output':>8s}")
print("-" * 75)

comparisons = [
    ("ollama", "llama3.2"),
    ("groq", "llama-3.3-70b-versatile"),
    ("groq", "llama-3.1-8b-instant"),
    ("together", "meta-llama/Llama-3.3-70B-Instruct-Turbo"),
    ("together", "meta-llama/Llama-3.1-8B-Instruct-Turbo"),
    ("vllm", "meta-llama/Llama-3.1-8B-Instruct"),
]

for provider, model_name in comparisons:
    config = load_provider_config(provider)
    meta = config.models.get(model_name)
    if meta:
        print(f"{provider:15s} {model_name:40s} ${meta.cost_input_per_1m:>6.2f}  ${meta.cost_output_per_1m:>6.2f}")

In [ ]:
# Capability comparison
print("=== Model Capabilities ===")
print(f"{'Provider':15s} {'Model':30s} {'Tools':>6s} {'Vision':>7s} {'Think':>6s} {'Context':>10s}")
print("-" * 80)

caps = [
    ("ollama", "llama3.2"),
    ("ollama", "deepseek-r1:8b"),
    ("groq", "llama-3.3-70b-versatile"),
    ("mistral", "mistral-large-latest"),
    ("mistral", "codestral-latest"),
    ("together", "deepseek-ai/DeepSeek-V3"),
]

for provider, model_name in caps:
    config = load_provider_config(provider)
    meta = config.models.get(model_name)
    if meta:
        print(f"{provider:15s} {model_name:30s} {str(meta.supports_tools):>6s} {str(meta.supports_vision):>7s} {str(meta.supports_thinking):>6s} {meta.context_window:>10,}")

---
## 12. Unknown Models — Graceful Defaults (D-103)

When a model isn't in the TOML, the adapter uses defaults from the config.

In [ ]:
# Load an unknown model
clear_cache()
model = load_model("ollama", model="some-new-model:latest")

print(f"Provider: {model.name}")
print(f"Model:    {model.model_name}")
print(f"Metadata: {model._model_meta}")
print("\nNo metadata = adapter uses defaults (4096 max_tokens, provider temperature).")
print("No error. Unknown models just work without metadata enrichment.")

---
## 13. Implementation Details

In [ ]:
print("=== BaseAdapter.__init__ (api_key_required check) ===")
from arcllm.adapters.base import BaseAdapter
print(inspect.getsource(BaseAdapter.__init__))

In [ ]:
print("=== OpenaiAdapter._build_headers (conditional auth) ===")
from arcllm.adapters.openai import OpenaiAdapter
print(inspect.getsource(OpenaiAdapter._build_headers))

In [ ]:
print("=== MistralAdapter (full with quirk overrides) ===")
print(inspect.getsource(MistralAdapter))

---
## Summary

| Component | What | Why |
|-----------|------|-----|
| 8 thin alias adapters | ~10 lines each, inherit OpenaiAdapter | OpenAI-compatible API is the standard |
| MistralAdapter | Quirk overrides for tool_choice + stop_reason | Only non-pure-alias provider |
| 9 provider TOMLs | Connection settings + model metadata | Config-driven, no code changes |
| `api_key_required` flag | Skips auth validation for local providers | Ollama/vLLM/TGI don't need keys |
| Zero cost defaults | `cost_*_per_1m = 0.0` for local | Local inference has no API cost |
| Dotted model names | Quoted keys in TOML (`"llama3.2"`) | TOML parsing gotcha |
| Convention-based registry | `{name}` → `arcllm.adapters.{name}` → `{Name}Adapter` | Zero registration code |
| Unknown model support | Adapter defaults when metadata missing | Graceful handling of new models |

**Usage**:
```python
# Local (no API key needed)
model = load_model("ollama")                    # Default: llama3.2
model = load_model("ollama", model="mistral")   # Specific model
model = load_model("vllm")                      # Self-hosted vLLM

# Cloud (API key required)
model = load_model("groq")                      # Fast inference
model = load_model("together")                  # Cost-optimized
model = load_model("fireworks")                 # Fast + tools
model = load_model("deepseek")                  # Reasoning
model = load_model("mistral")                   # EU sovereignty
model = load_model("huggingface")               # HF Inference API
model = load_model("huggingface_tgi")           # Self-hosted TGI
```

**Test count**: 538 passing (83 new: 69 parametrized in `test_open_providers.py` + 14 in `test_mistral.py`)